# Algoritmos de Reparto Proporcional

En primer lugar programaremos el algoritmo para obtener la solución mediante los métodos de cuotas y restos mayores, y después se programará el algoritmo para obtener la solución mediante los métodos de divisores.

Importamos las diferentes funciones de las librerías de python:

- `sympy` para poder utilizar fracciones e infinito.

- `numpy` para poder utilizar matrices y vectores.

- la función `combinations` de la librería `itertools`.

- la función `Fraction` de la librería `fractions`.

- las funciones `Problem` y `ExactSumConstraint` de la librería `constraint`.

In [4]:
from sympy import *
from numpy import *
#from collections import Counter
from itertools import combinations
from fractions import Fraction
from constraint import Problem, ExactSumConstraint

En primer lugar es necesario obtener una función que encuentre el máximo de un vector, ya que serán los elementos que obtengan el máximo valor del vector a quienes asignemos un escaño, tanto en los métodos de cuotas y restos mayores como en los métodos de divisores.

In [5]:
def BuscaMaximo(v):
    """
    args: (v)donde
    v es una lista o array.
    
    Función para calcular el máximo de un vector (o matriz) "v". 
    
    return:(maximo,repeticiones,posiciones) donde 
    maximo es el máximo del vector, 
    repeticiones es el número de posiciones de v en las que se encuentra y 
    posiciones es el vector de posiciones de v en las que se encuentra dicho mínimo.
    
    Ejemplo:
    >>> v = [1,4,2,5,8,3]
    >>> M,npos,pos=BuscaMaximo(v)
    >>> M
    """
    maximo=v[0] #esta variable guardará el máximo temporal
    repeticiones=1 #esta variable guardará el número de veces que se repite el máximo temporal
    posiciones=[0] #esta variable almacenará las posiciones en las que se encuentra el máximo temporal
    for i in range(1,len(v)): #recorremos el vector en busca de un máximo temporal más grande o más repeticiones
        if(v[i]==maximo): #si se encuentra una repetición del máximo temporal que se añada una repetición y la posición de esta
            repeticiones+=1
            posiciones+=[i]
        else:
            if(v[i]>maximo): #si se encuentra un nuevo máximo temporal que se añada a la variable maximo y se reseteen las otras variables
                maximo=v[i]
                repeticiones=1
                posiciones=[i]
    return(maximo,repeticiones,posiciones)

Python tiene ya implementada y optimizada una función que determina el máximo de un vector, incluso de una matriz, incluida en la librería numpy para elementos de tipo array. Por tanto, implementaremos una función que utilice dicho método para encontrar el máximo de una lista.

In [6]:
def BuscaMaximo2(v): 
    """
    args: (v)donde
    v es una lista, lista de listas o array.
    
    Función para calcular el máximo de un vector (o matriz) "v" utilizando el comando array. Es más rápida que BuscaMaximo.
    
    return:(maximo,num_pos,pos) donde 
    maximo es el máximo del vector, 
    num_pos es el número de posiciones de v en las que se encuentra y 
    pos es el vector de posiciones de v en las que se encuentra dicho mínimo.
    
    Ejemplo:
    >>> v = [1,4,2,5,8,3]
    >>> M,npos,pos=BuscaMaximo2(v)
    >>> M
    """
    v1=array(v)
    maximo=v1.max()
    posiciones=list(where(v1==maximo)[0])
    return(maximo,len(posiciones),posiciones) 

Para el caso en que haya algún empate, es necesaria una función que realice todas las posibles combinaciones de determinado tamaño de una lista.

In [7]:
def CombinacionesLista(v,n,inicial=[]): 
    """
    args: (v, n, [inicial])donde
    v es una lista, lista de listas o array,
    n es un entero no negativo y
    inicial (opcional) es una lista
    
    Función para obtener las posibles sublistas de una lista (o matriz unidimensional) "v" de dimensión n y añadirlas al final de la matriz inical.
    
    return:(comb) donde 
    comb es una lista de las posibles sublistas de tamaño n añadidas a inicial.
    
    Ejemplo:
    >>> v = [1,4,2,5,8,3]
    >>> c=CombinacionesLista(v,3)
    >>> c
    """
    if (len(v)<n):
        comb=[]
    elif(n == 0): #si el tamaño es 0, tan solo se puede poner el vector inicial.
        comb = [inicial]
    else: #en caso de que el tamaño no sea 0, las combinaciones se construyen seleccionando un elemento y construyendo las combinaciones de tamaño n-1 de la parte de la derecha de ese elemento.
        l = len(v)
        comb = []
        for i in range(l):
            comb+=CombinacionesLista(v[i + 1 : l], n - 1,  inicial + [v[i]])
    return (comb)

Sin embargo, al igual que en el caso de encontrar el máximo, ya hay una función implementada en la librería itertools que realiza todas las posibles combinaciones de tamaño n de una lista o un array unidimensional, utilizaremos esta para hacer una función más optimizada para obtener las combinaciones de tamaño n de una lista.

In [8]:
def CombinacionesLista2(v,n):
    """
    args: (v, n)donde
    v es una lista, lista de listas o array y
    n es un entero no negativo
    
    Función para obtener las posibles sublistas de una lista (o matriz unidimensional) "v" de dimensión n.
    
    return:(comb) donde 
    comb es una lista de las posibles sublistas de tamaño n.
    
    Ejemplo:
    >>> v = [1,4,2,5,8,3]
    >>> c=CombinacionesLista2(v,3)
    >>> c
    """
    return (list(combinations(v,n)))

## Métodos de cuotas y restos mayores

Ya estamos en disposición de crear una función que devuelva el reparto (o repartos en caso de que haya empates) por el método de Restos Mayores. En primer lugar, se utilizarán las funciones hechas a mano y en segundo lugar las ya implementadas en python para observar la diferencia de tiempo entre ellas, por lo que en adelante sólo se utilizarán las funciones optimizadas.

In [9]:
def RestosMayores (v,H,k=0):
    """
    args: (v, H, [k])donde
    v es una lista o array unidimensional,
    H es un entero no negativo y
    k (opcional) es un número cualquiera (generalmente en coma flotante). Toma el valor por defecto correspondiente al reparto por el método de Hamilton.
    
    Posibles valores para k:
    - H/sum(v) (Hamilton)
    - (H+1)/sum(v) (Droop)
    - (H+1)/sum(v) (Imperiali)
    
    Función que hace el reparto de restos mayores con vector de votos v, escaños a repartir H y factor k si el factor k no se da o es menor o igual que 0 se supone que se realiza el método de Hamilton.
       
    return:(repartos) donde 
    repartos es una lista de los posibles repartos que se admite con vector de votos v, escaños a repartir H y factor k.
    
    Ejemplo:
    >>> v = [100,125,150,200]
    >>> rmc=RestosMayores(v,26)
    >>> rm
    """
    
    if(k<=0):
        k=H/(sum(v)*1.0)
    q=v*1.0*k
    h=[] #e será el vector que almacene el número de escaños a cada partido.
    d=[] #d será el vector que almacene los restos de cada partido
    por_repartir=H #almacena el número de escaños que no se han asignado
    for i in range(len(q)):
        h=h+[floor(q[i])] #a cada partido se le da la cuota inferior en escaños
        por_repartir-=h[i] #se resta el número de escaños dados
        d=d+[q[i]-h[i]] #y se calcula los restos del partido i
    while(por_repartir>0):#mientras no se repartan todos los escaños que se haga lo siguiente:
        ma,rep,pos=BuscaMaximo(d) #Busca el máximo del vector d, el número de repeticiones de este y las posiciones que ocupan
        if(rep<=por_repartir): #Si hay suficientes escaños para darle uno a cada repetición que se lo dé y que ponga el resto en 0
            por_repartir-=rep
            for j in pos:
                h[j]+=1
                d[j]=0
        else: #en caso de que no haya suficientes escaños ya no quedarán más escaños que repartir despues (por_repartir<0) y además habrá que dar varias soluciones
            combinaciones=CombinacionesLista(pos,por_repartir) #comenzamos haciendo las posibles combinaciones de los subindices que tienen el máximo resto
            repartos=[] #creamos un vector que será el que aloje todos los posibles resultados
            h1=[] #este vector contendrá uno a uno los resultados finales
            for combi in combinaciones: #recorreo las posibles combinaciones de subindices
                h1[:]=h[:] #copia en h1 el vector de reparto que teníamos a falta de repartir los escaños que no son suficientes para el empate
                for indi in combi:
                    h1[indi]+=1 #añade un escaño a los elementos de h1 que tienen subindice en la combinación en la que estamos
                repartos+=[h1[:]] #añade h1 al vector de repartos
            por_repartir-=rep
    if(por_repartir==0): #si por_repartir=0, entonces no se ha creado un vector de soluciones, sino la solución, por tanto, para que la devolución sea más uniforme creamos un vector con la solución
        repartos=[h]
    return(repartos)

In [10]:
def RestosMayores2 (v,H,k=0):
    """
    args: (v, H, [k])donde
    v es una lista o array unidimensional,
    H es un entero no negativo y
    k (opcional) es un número cualquiera (generalmente en coma flotante). Toma el valor por defecto correspondiente al reparto por el método de Hamilton.
    
    Posibles valores para k:
    - H/sum(v) (Hamilton)
    - (H+1)/sum(v) (Droop)
    - (H+1)/sum(v) (Imperiali)
    
    Función que hace el reparto de restos mayores con vector de votos v, escaños a repartir H y factor k si el factor k no se da o es menor o igual que 0 se supone que se realiza el método de Hamilton.
    
    Esta función utiliza las funciones ya implementadas en python para encontrar el máximo y las posibles combinaciones de sublistas de tamaño n de una lista.
    
    return:(repartos) donde 
    repartos es una lista de los posibles repartos que se admite con vector de votos v, escaños a repartir H y factor k.
    
    Ejemplo:
    >>> v = [100,125,150,200]
    >>> rmc=RestosMayores2(v,26)
    >>> rm
    """
    if(k<=0):
        k=H/(sum(v)*1.0)
    q=v*1.0*k
    h=[] #e será el vector que almacene el número de escaños a cada partido.
    d=[] #d será el vector que almacene los restos de cada partido
    por_repartir=H #almacena el número de escaños que no se han asignado
    for i in range(len(q)):
        h=h+[floor(q[i])] #a cada partido se le da la cuota inferior en escaños
        por_repartir-=h[i] #se resta el número de escaños dados
        d=d+[q[i]-h[i]] #y se calcula los restos del partido i
    while(por_repartir>0):#mientras no se repartan todos los escaños que se haga lo siguiente:
        ma,rep,pos=BuscaMaximo2(d) #Busca el máximo del vector d, el número de repeticiones de este y las posiciones que ocupan
        if(rep<=por_repartir): #Si hay suficientes escaños para darle uno a cada repetición que se lo dé y que ponga el resto en 0
            por_repartir-=rep
            for j in pos:
                h[j]+=1
                d[j]=0
        else: #en caso de que no haya suficientes escaños ya no quedarán más escaños que repartir despues (por_repartir<0) y además habrá que dar varias soluciones
            combinaciones=CombinacionesLista2(pos,por_repartir) #comenzamos haciendo las posibles combinaciones de los subindices que tienen el máximo resto
            repartos=[] #creamos un vector que será el que aloje todos los posibles resultados
            h1=[] #este vector contendrá uno a u\no los resultados finales
            for combi in combinaciones: #recorreo las posibles combinaciones de subindices
                h1[:]=h[:] #copia en h1 el vector de reparto que teníamos a falta de repartir los escaños que no son suficientes para el empate
                for indi in combi:
                    h1[indi]+=1 #añade un escaño a los elementos de h1 que tienen subindice en la combinación en la que estamos
                repartos+=[h1[:]] #añade h1 al vector de repartos
            por_repartir-=rep
    if(por_repartir==0): #si por_repartir=0, entonces no se ha creado un vector de soluciones, sino la solución, por tanto, para que la devolución sea más uniforme creamos un vector con la solución
        repartos=[h]
    return(repartos)

## Métodos divisores

La categoría más utilizada de métodos de reparto proporcional es la categoría de métodos divisores. A continuación se programa el algoritmo para encontrar la solución (o soluciones) mediante estos métodos. En primer lugar se utilizarán las funciones para buscar el máximo y las combinaciones de sublistas de tamaño n de una lista que se han programado y después se usarán las ya implementadas en python.

In [11]:
def Divisores (v,H,d=lambda x: x+1):
    """
    args: (v, H, [d])donde
    v es una lista o array unidimensional,
    H es un entero no negativo y
    d (opcional) es un la función divisora (generalmente una función de tipo lambda(x)). Toma el valor por defecto correspondiente al reparto por el método D'Hondt.
    
    Posibles valores para d(x):
    - x+1 (D´Hondt)
    - x+0.5 (Sainte-Lagüe)
    - x (Adams)
    - ...
    
    Función que hace el reparto por el método de divisores con función divisora d, vector de votos v y escaños a repartir H.
    
    return:(repartos, cocientes) donde 
    repartos es una lista de los posibles repartos que se admite con vector de votos v, escaños a repartir H y función divisora d
    
    Ejemplo:
    >>> v = [100,125,150,200]
    >>> dv=Divisores(v,26)
    >>> dv
    """
    
    divisores=[d(0)]#crea un vector que almacene los divisores de momento tendrá el primer valor de la función divisora.
    h=[] #e será el vector que almacene el número de escaños a cada partido.
    q=[] #d será el vector que almacene los cocientes para el divisor del número de escaños que se le haya dado.
    for i in range(len(v)):
        h+=[0]
        if divisores[0]==0:
            q+=[oo]
        else:
            q+=[v[i]/(1.0*divisores[0])]
    por_repartir=H #almacena el número de escaños que no se han asignado
    while(por_repartir>0):#mientras no se repartan todos los escaños que se haga lo siguiente:
        ma,rep,pos=BuscaMaximo(q) #Busca el máximo del vector d, el número de repeticiones de este y las posiciones que ocupan
        if(rep<=por_repartir): #Si hay suficientes escaños para darle uno a cada repetición que se lo dé y que cambie el cociente asociado a ese partido
            por_repartir-=rep
            for j in pos:
                h[j]+=1
                if(h[j]==len(divisores)):
                    divisores+=[d(h[j])]
                q[j]=v[j]/(1.0*divisores[h[j]])
        else: #en caso de que no haya suficientes escaños ya no quedarán más escaños que repartir despues (por_repartir<0) y además habrá que dar varias soluciones
            combinaciones=CombinacionesLista(pos,por_repartir) #comenzamos haciendo las posibles combinaciones de los subindices que tienen el máximo resto
            repartos=[] #creamos un vector que será el que aloje todos los posibles resultados
            h1=[] #este vector contendrá uno a uno los resultados finales
            for combi in combinaciones: #recorre las posibles combinaciones de subindices
                h1[:]=h[:] #copia en h1 el vector de reparto que teníamos a falta de repartir los escaños que no son suficientes para el empate
                for indi in combi:
                    h1[indi]+=1 #añade un escaño a los elementos de h1 que tienen subindice en la combinación en la que estamos
                repartos+=[h1[:]] #añade h1 al vector de repartos
            por_repartir-=rep
    if(por_repartir==0): #si por_repartir=0, entonces no se ha creado un vector de soluciones, sino la solución, por tanto, para que la devolución sea más uniforme creamos un vector con la solución
        repartos=[h]
    return(repartos)

Ahora se programará la función Divisores usando las funciones ya implementadas en python.

In [12]:
def Divisores2 (v,H,d=lambda x: x+1):
    """
    args: (v, H, [d])donde
    v es una lista o array unidimensional,
    H es un entero no negativo y
    d (opcional) es un la función divisora (generalmente una función de tipo lambda(x)). Toma el valor por defecto correspondiente al reparto por el método D'Hondt.
    
    Posibles valores para d(x):
    - x+1 (D´Hondt)
    - x+0.5 (Sainte-Lagüe)
    - x (Adams)
    - ...
    
    Función que hace el reparto por el método de divisores con función divisora d, vector de votos v y escaños a repartir H.
    
    Esta función utiliza las funciones ya implementadas en python para encontrar el máximo y las posibles combinaciones de sublistas de tamaño n de una lista.
    
    return:(repartos) donde 
    repartos es una lista de los posibles repartos que se admite con vector de votos v, escaños a repartir H y función divisora d
    
    Ejemplo:
    >>> v = [100,125,150,200]
    >>> dv=Divisores2(v,26)
    >>> dv
    """
    divisores=[d(0)]#crea un vector que almacene los divisores de momento tendrá el primer valor de la función divisora.
    h=[] #e será el vector que almacene el número de escaños a cada partido.
    q=[] #d será el vector que almacene los cocientes para el divisor del número de escaños que se le haya dado.
    for i in range(len(v)):
        h+=[0]
        if divisores[0]==0:
            q+=[oo]
        else:
            q+=[v[i]/(1.0*divisores[0])]
    por_repartir=H #almacena el número de escaños que no se han asignado
    while(por_repartir>0):#mientras no se repartan todos los escaños que se haga lo siguiente:
        ma,rep,pos=BuscaMaximo2(q) #Busca el máximo del vector d, el número de repeticiones de este y las posiciones que ocupan
        if(rep<=por_repartir): #Si hay suficientes escaños para darle uno a cada repetición que se lo dé y que cambie el cociente asociado a ese partido
            por_repartir-=rep
            for j in pos:
                h[j]+=1
                if(h[j]==len(divisores)):
                    divisores+=[d(h[j])]
                q[j]=v[j]/(1.0*divisores[h[j]])
        else: #en caso de que no haya suficientes escaños ya no quedarán más escaños que repartir despues (por_repartir<0) y además habrá que dar varias soluciones
            combinaciones=CombinacionesLista2(pos,por_repartir) #comenzamos haciendo las posibles combinaciones de los subindices que tienen el máximo resto
            repartos=[] #creamos un vector que será el que aloje todos los posibles resultados
            h1=[] #este vector contendrá uno a uno los resultados finales
            for combi in combinaciones: #recorre las posibles combinaciones de subindices
                h1[:]=h[:] #copia en h1 el vector de reparto que teníamos a falta de repartir los escaños que no son suficientes para el empate
                for indi in combi:
                    h1[indi]+=1 #añade un escaño a los elementos de h1 que tienen subindice en la combinación en la que estamos
                repartos+=[h1[:]] #añade h1 al vector de repartos
            por_repartir-=rep
    if(por_repartir==0): #si por_repartir=0, entonces no se ha creado un vector de soluciones, sino la solución, por tanto, para que la devolución sea más uniforme creamos un vector con la solución
        repartos=[h]
    return(repartos)

In [13]:
Divisores2([1,1,1],2)

[[1, 1, 0], [1, 0, 1], [0, 1, 1]]

# Algoritmo de reparto Biproporcional

Para obtener la solución de un reparto biproporcional de un problema idénticamente constreñido se programa el único algoritmo determinista que se conoce hasta el momento, el algoritmo Tie and Transfer, diseñado para dicho caso concreto.

Para ello, además de reutilizar la función BuscaMaximo2, crearemos una función similar para encontrar el mínimo:

In [14]:
def BuscaMinimo2(v):
    """
    args: (v) donde
    v es una lista, lista de listas o array.
    
    Función para calcular el mínimo de un vector (o matriz) "v" utilizando el comando array. Es más rápida que BuscaMinimo
    
    return:(minimo,num_pos,pos) donde 
    minimo es el mínimo del vector, 
    num_pos es el número de posiciones de v en las que se encuentra y 
    pos es el vector de posiciones de v en las que se encuentra dicho mínimo.
    
    Ejemplo:
    >>> v = [1,4,2,5,8,3]
    >>> m,num_pos,pos=BuscaMinimo2(v)
    >>> m
    """
    v1=array(v)
    minimo=v1.min()
    posiciones=list(where(v1==minimo)[0])
    return(minimo,len(posiciones),posiciones)

Para programar el algoritmo Tie and Transfer, es necesario hacer una función que proporcione el reparto por un método divisor con función divisora `d` y escaños a repartir `H` que, en caso de empate, reparta los escaños sobrantes de forma arbitraria, por ejemplo, de abajo a arriba. Además, esta función deberá proporcionar datos adicionales del reparto: los últimos cocientes que han recibido escaño por cada posición y los primeros que recibirían, y las posiciones en situación de empate y en cuáles de ellas se ha concedido este.

In [15]:
def Divisores4 (v,H,d=lambda x:x+1):
    """
    args: (v,H,[d]) donde
    v es una lista o array unidimensional,
    H es un número entero no negativo y
    d (opcional) predefinido como lambda(x):x+1 es la función correspondiente a la función divisora (predefinida como la correspondiente a D'Hondt).
    
    Función para calcular el reparto por el método de divisores con vector de votos v, escaños a repartir H y función divisora d.
    Difiere del resto de funciones del método de divisores en que reparte los escaños que encuentra en posición de empate de abajo a arriba.
    
    return:(h,ultimos,primeros,empates,concedidos) donde 
    h es el vector de reparto,
    ultimos es el vector de cuotas,
    primeros es el vector de cuotas de un escaño más,
    empates es el vector binario que determina en qué posiciones hay empate y
    concedidos es el vector binario que determina a qué posiciones en empate se ha concedido un escaño.
    
    Ejemplo:
    >>> v = [1000,500,250,125]
    >>> h,ultimos,primeros,empates,concedidos=Divisores4(v,15)
    >>> h
    """
    divisores=[d(0)]#crea un vector que almacene los divisores de momento tendrá el primer valor de la función divisora.
    h=[] #e será el vector que almacene el número de escaños a cada partido.
    q=[] #d será el vector que almacene los cocientes para el divisor del número de escaños que se le haya dado.
    empates=[]
    concedidos=[]
    for i in range(len(v)):
        h+=[0]
        empates+=[0]
        concedidos+=[0]
        if divisores[0]==0:
            q+=[oo]
        else:
            q+=[v[i]/(1.0*divisores[0])]
    por_repartir=H #almacena el número de escaños que no se han asignado
    while(por_repartir>0):#mientras no se repartan todos los escaños que se haga lo siguiente:
        ma,rep,pos=BuscaMaximo2(q) #Busca el máximo del vector d, el número de repeticiones de este y las posiciones que ocupan
        if(rep<=por_repartir): #Si hay suficientes escaños para darle uno a cada repetición que se lo dé y que cambie el cociente asociado a ese partido
            por_repartir-=rep
            for j in pos:
                h[j]+=1
                if(h[j]==len(divisores)):
                    divisores+=[d(h[j])]
                q[j]=v[j]/(1.0*divisores[h[j]])
        else: #en caso de que no haya suficientes escaños ya no quedarán más escaños que repartir despues (por_repartir<0) y además habrá que dar varias soluciones
            for j in pos:
                empates[j]=1
            for j in range(por_repartir):
                h[pos[-j-1]]+=1
                concedidos[pos[-j-1]]=1
                if(h[pos[-j-1]]==len(divisores)):
                    divisores+=[d(h[pos[-j-1]])]
                q[pos[-j-1]]=v[pos[-j-1]]/(1.0*divisores[h[pos[-j-1]]])
            por_repartir-=rep
        repartos=h
    ultimos=[]
    primeros=[]
    for i in range(len(h)):
        if divisores[h[i]]==0:
            primeros+=[oo]
        else:
            primeros+=[v[i]/divisores[h[i]]]
        if(h[i]>=1) and divisores[h[i]-1]!=0:
            ultimos+=[v[i]/divisores[h[i]-1]]
        else:
            ultimos+=[oo]
    return(h,ultimos,primeros,empates,concedidos)

También necesitaremos para programar el algoritmo Tie and Transfer una función que construya el grafo de acuerdo al propio algoritmo a partir de la matriz de empates, la matriz de empates concedidos y los conjuntos I+ I- e I0. Un grafo dirigido se puede considerar como un diccionario cuyas llaves son los puntos de partida de las aristas y los argumentos de cada llave son una lista de puntos de llegada de las aristas que parten de la llave correspondiente.

In [16]:
def construir_grafo(empates,empates_dados,Imas,Imenos,I0):
    """
    args: (empates,empates_dados, Imas, Imenos, I0) donde
    empates es un array array bidimensional binario correspondiente a las posiciones de empate,
    empates_dados es un array bidimensional (o lista de listas) binario correspondiente a las posiciones de los empates concedidos
    Imas es una lista con los índices de las filas sobrerepresentadas
    Imenos es una lista con los índices de las filas infrarepresentadas e
    I0 es una lista con los índices de las filas bien representadas
     
    Función para construir el grafo del algoritmo Tie and Transfer para el caso idénticamente constreñido con matriz de empates "empates", matriz de empates concedidos "empates_dados" y filas sobrerepresentadas, bien representadas e infrarepresentadas Imas, I0 e Imenos respectivamente.
    
    return:(grafo) donde 
    grafo es un diccionario, correspondiente al grafo donde 
    ultimos es el vector de cuotas,
    primeros es el vector de cuotas de un escaño más,
    empates es el vector binario que determina en qué posiciones hay empate y
    concedidos es el vector binario que determina a qué posiciones en empate se ha concedido un escaño.
    
    Ejemplo:
    >>> e = [[1,1,1,0,1],[0,1,0,1,0],[1,0,1,0,0],[1,0,0,1,1]]
    >>> ed = [[1,0,1,0,0],[0,1,0,1,0],[1,0,0,0,0],[0,0,0,0,1]]
    >>> Imas = [0,1]
    >>> Imenos = [2,3,4]
    >>> I0 = []
    >>> grafo=construir_grafo(e,ed,Imas,Imenos,I0)
    >>> grafo
    """
    sz=empates.shape
    grafo={}
    for i in Imas:
        for j in range(sz[1]):
            if empates[i,j]==1 and empates_dados[i,j]==1:
                if ("f"+str(i) in grafo):
                    grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                else:
                    grafo["f"+str(i)]=["c"+str(j)]
    for i in Imenos:
        for j in range(sz[1]):
            if empates[i,j]==1 and empates_dados[i,j]==0:
                if ("c"+str(j) in grafo):
                    grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                else:
                    grafo["c"+str(j)]=["f"+str(i)]
    for i in I0:
        for j in range(sz[1]):
            if empates[i,j]==1:
                if  empates_dados[i,j]==0:
                    if ("c"+str(j) in grafo):
                        grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                    else:
                        grafo["c"+str(j)]=["f"+str(i)]
                else:
                    if ("f"+str(i) in grafo):
                        grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                    else:
                        grafo["f"+str(i)]=["c"+str(j)]
    return(grafo)

También necesitaremos un algoritmo que encuentre un camino entre dos nodos tales que el nodo inicial pertenezca a un conjunto de partida y el final pertenezca a uno de llegada. Para ello, hemos modificado la función find_path de la página web https://www.python.org/doc/essays/graphs/ para que admita un conjunto de puntos de partida y de llegada.

In [17]:
def find_path_modified0 (graph, start, end, checked=[], path=[]):
    """
    args: (graph, start, end, [checked], [path]) donde
    graph es un diccionario correspondiente al grafo en el que se desea encontrar el camino,
    start es una lista que contiendeel conjunto de nodos de inicio,
    end es una lista que contiene el conjunto de nodos de finalización,
    checked (opcional) es el conjunto de nodos que no se desea que se comprueben y
    path (opcional) es una lista de nodos a la que se unirán los nodos del camino.
     
    Función para encontrar un camino entre dos nodos del grafo graph tales que el nodo de partida pertenezca al conjunto start y el de fin al conjunto end que no pase por los nodos checked y que añada el camino a la variable path.
    
    return:(grafo) donde 
    grafo es un diccionario, correspondiente al grafo donde 
    ultimos es el vector de cuotas,
    primeros es el vector de cuotas de un escaño más,
    empates es el vector binario que determina en qué posiciones hay empate y
    concedidos es el vector binario que determina a qué posiciones en empate se ha concedido un escaño.
    
    Ejemplo:
    >>> e = [[1,1,1,0,1],[0,1,0,1,0],[1,0,1,0,0],[1,0,0,1,1]]
    >>> ed = [[1,0,1,0,0],[0,1,0,1,0],[1,0,0,0,0],[0,0,0,0,1]]
    >>> Imas = [0,1]
    >>> Imenos = [2,3,4]
    >>> I0 = []
    >>> grafo = construir_grafo(e,ed,Imas,Imenos,I0)
    >>> path = find_path_modified0 (graph, ["f0","f1"], ["f2","f3","f4"])
    >>> path
    """
    #comprueba si el principio del camino será por defecto o no.
    camino=path+[start[0]]
    new_checked=checked+[start[0]] # añado el primer elemento de los posibles comienzos (nodo en el que se encuentra comprobando) a comprobados
    if start[0] in end: #si el nodo está en el final ya se ha encontrado el principio y el final.
        return camino
    if not start[0] in graph: #si el nodo no está en el grafo (no es un nodo de partida), si el el principio del camino ovbiarlo (al final de la función pasa a otro comienzo de camino) y si no devolver nada para que continúe con otro nodo
        if start[0]!=camino[0]:
            return None
    else: #Si el nodo está en el grafo comprobará uno a uno si encuentra un camino que llegue a alguno de los nodos de "end" desde
        for node in graph[start[0]]:
            if node not in new_checked:
                new_path=find_path_modified(graph,[node]+start[1:],end, new_checked, camino)
                if new_path: 
                    return new_path
    if len(start)>1:
        return find_path_modified(graph, start[1:], end, new_checked)
    else:
        return None

Además, hay ocasiones en las que interesa que la función devuelva qué aristas ha comprobado o no se le permitía comprobar. Por ello, se ha vuelto a modificar la función find_path_modified0 para que devuelva una lista con los nodos comprobados.

In [18]:
def find_path_modified (graph, start, end, checked=[], path=[]):
    """
    args: (graph, start, end, [checked], [path]) donde
    graph es un diccionario correspondiente al grafo en el que se desea encontrar el camino,
    start es una lista que contiendeel conjunto de nodos de inicio,
    end es una lista que contiene el conjunto de nodos de finalización,
    checked (opcional) es el conjunto de nodos que no se desea que se comprueben y
    path(opcional) es una lista de nodos a la que se unirán los nodos del camino.
     
    Función para encontrar un camino entre dos nodos del grafo graph tales que el nodo de partida pertenezca al conjunto start y el de fin al conjunto end que no pase por los nodos checked y que añada el camino a la variable path.
    
    return:(grafo) donde 
    grafo es un diccionario, correspondiente al grafo donde 
    ultimos es el vector de cuotas,
    primeros es el vector de cuotas de un escaño más,
    empates es el vector binario que determina en qué posiciones hay empate y
    concedidos es el vector binario que determina a qué posiciones en empate se ha concedido un escaño.
    
    Ejemplo:
    >>> e = [[1,1,1,0,1],[0,1,0,1,0],[1,0,1,0,0],[1,0,0,1,1]]
    >>> ed = [[1,0,1,0,0],[0,1,0,1,0],[1,0,0,0,0],[0,0,0,0,1]]
    >>> Imas = [0,1]
    >>> Imenos = [2,3,4]
    >>> I0 = []
    >>> grafo=construir_grafo(e,ed,Imas,Imenos,I0)
    >>> path, compr= find_path_modified0 (graph, ["f0","f1"], ["f2","f3","f4"])
    >>> path
    """
    #comprueba si el principio del camino será por defecto o no.
    camino=path[:]+[start[0]]
    if start[0] not in checked:
        new_checked=checked[:]+[start[0]] # añado el primer elemento de los posibles comienzos (nodo en el que se encuentra comprobando) a comprobados
    else:
        new_checked=checked[:]
    if start[0] in end: #si el nodo está en el final ya se ha encontrado el principio y el final.
        return (camino,new_checked)
    if not start[0] in graph: #si el nodo no está en el grafo (no es un nodo de partida), si el el principio del camino ovbiarlo (al final de la función pasa a otro comienzo de camino) y si no devolver nada para que continúe con otro nodo
        if start[0]!=camino[0]:
            return (None, new_checked)
    else: #Si el nodo está en el grafo comprobará uno a uno si encuentra un camino que llegue a alguno de los nodos de "end" desde
        for node in graph[start[0]]:
            if node not in new_checked:
                new_path, new_checked = find_path_modified(graph,[node]+start[1:],end, new_checked, camino)
                if new_path: 
                    return (new_path,new_checked)
    if len(start)>1:
        return find_path_modified(graph, start[1:], end, new_checked)
    else:
        return (None,new_checked)

In [19]:
def DivisoresSinEmpates (v,H,d=lambda x: x+1):
    """
    args: (v,H,[d]) donde
    v es una lista o array unidimensional,
    H es un número entero no negativo y
    d (opcional) predefinido como lambda(x):x+1 es la función correspondiente a la función divisora (predefinida como la correspondiente a D'Hondt).
    
    Función para calcular el reparto por el método de divisores con vector de votos v, escaños a repartir H y función divisora d.
    Difiere del resto de funciones del método de divisores en que no reparte los escaños que encuentra en posición de empate, sino que devuelve las posiciones que están en situación de empate y la cantidad de escaños sobrantes.
    También devuelve el último cociente que ha obtenido un escaño.
    
    return:(h,ma,pos,sobrantes) donde 
    h es el vector de reparto,
    ma es un entero correspondiente al último cociente en recibir un escaño,
    pos es el vector binario que determina en qué posiciones hay empate y
    sobrantes es un entero correspondiente al número de escaños a repartir entre los empates.
    
    Ejemplo:
    >>> v = [1000,500,250,125]
    >>> h,ma,pos,sobrantes = DivisoresSinEmpates(v,15)
    >>> h
    """
    a=array(v)
    divisores=[d(0)] #crea un vector que almacene los divisores de momento tendrá el primer valor de la función divisora.
    h=[] #e será el vector que almacene el número de escaños a cada partido.
    q=[] #d será el vector que almacene los cocientes para el divisor del número de escaños que se le haya dado.
    for i in range(len(v)):
        h+=[0]
        if divisores[0]==0:
            q+=[oo]
        else:
            q+=[v[i]/(divisores[0])]
    sobrantes=0
    por_repartir=H #almacena el número de escaños que no se han asignado
    while(por_repartir>0):#mientras no se repartan todos los escaños que se haga lo siguiente:
        ma,rep,pos=BuscaMaximo2(q) #Busca el máximo del vector d, el número de repeticiones de este y las posiciones que ocupan
        if(rep<=por_repartir): #Si hay suficientes escaños para darle uno a cada repetición que se lo dé y que cambie el cociente asociado a ese partido
            por_repartir-=rep
            for j in pos:
                h[j]+=1
                if(h[j]==len(divisores)):
                    divisores+=[d(h[j])]
                q[j]=v[j]/(divisores[h[j]])
        else:
            sobrantes=por_repartir
            por_repartir=0
    return(h,ma,pos,sobrantes)

In [20]:
DivisoresSinEmpates([ 2.,  1.5,  2.25,  3.,    1.  ],5)

([1, 1, 1, 2, 0], 1.5, [1, 3], 0)

Finalmente, necesitaremos una función en la que, dada una matriz de posiciones en situación de empate y la cantidad de empates que hay que conceder por filas y columnas junto a la matriz de reparto sin empates, reparta los empates de forma que sean compatibles con las restricciones de las filas y columnas. Para ello, utilizamos las funciones de la librería constraint. Se pueden encontrar ejemplos de uso de esta librería en las web https://labix.org/python-constraint y https://pypi.python.org/pypi/python-constraint .

In [21]:
def matrix_to_problem(m,r,c,a):
    """
    args: (m,r,c[,a]) donde
    m es un array bidimensional binario que representará la matriz de empates,
    r es una lista o array unidimensional en el que cada posición i contiene el número de escaños a repartir entre los empates que hay en la fila i,
    c es una lista o array unidimensional en el que cada posición j contiene el número de escaños a repartir entre los empates que hay en la columna j y
    a es un array bidimensional o una lista de listas correspondiente a la matriz de reparto de un problema sin conceder ningún empate. Por defecto valdrá una matriz de ceros de la misma dimensión que m.
    
    Función para resolver el problema de constricción bidimensional con matriz de votos "m" y restricciones por filas y columnas "r" y "c" respectivamente incluyendo la restricción de que la matriz resultante debe ser binaria.
    Además, esta función incluye un argumento "a" correspondiente a una matriz a la que se sumarán los resultados de dicho reparto.
    
    return:(sols) donde 
    sols es una lista que contiene los distintos arrays resultantes de sumar a "a" las diversas soluciones del problema de constricción.
    
    Ejemplo:
    >>> v = array([[1,0,0,1],[1,1,1,0],[0,1,1,0],[1,0,0,0],[0,0,1,1],[0,0,0,1]])
    >>> r = [1,2,1,1,1,1]
    >>> c = [1,2,2,2]
    >>> s = matrix_to_problem(v,r,c,zeros(v.shape))
    >>> s
    """
    sz = m.shape
    p = Problem()
    tv =[] #vector que contendrá las variables de la matriz
    for i in range(sz[0]): #añadir variables del interior de la matriz que son distintas de cero
        for j in range(sz[1]):
            if m[i][j]!=0:
                p.addVariable('x'+str(i)+"_"+str(j),range(2))
                tv.append('x'+str(i)+"_"+str(j))
    #print("Variables involved ",tv)
    for i in range(sz[0]): #por filas hacer: 
        nz = [j for j in range(sz[1]) if m[i][j]!=0] #obtener el índice de la columna de los elementos de la fila que son distintos de 0
        vs = ['x'+str(i)+"_"+str(j) for j in nz] #obtener las variables del interior de la matriz que no valen 0 en la columna i
        if vs!=[]:
            #print("adding sum of "+str(vs)+"="+str(r[i]))
            p.addConstraint(ExactSumConstraint(r[i]),vs) #añade al problema la restricción de la suma de la fila i.
    for j in range(sz[1]): #lo mismo que por filas para las columnas
        nz = [i for i in range(sz[0]) if m[i][j]!=0]
        vs = ['x'+str(i)+"_"+str(j) for i in nz]
        if vs!=[]:
            #print("adding sum of "+str(vs)+"="+str(c[j]))
            p.addConstraint(ExactSumConstraint(c[j]),vs)
    sols = []#crea el vector de soluciones
    for s in p.getSolutions(): #resuelve el problema y rellena las matrices con los datos del problema.
        #print(s)
        mc = a.copy()
        for i in range(sz[0]):
             for j in range(sz[1]):
                if m[i][j]==1:
                    mc[i][j]=s["x"+str(i)+"_"+str(j)]+a[i][j]
        #print(mc)
        sols.append(mc)
    return sols

Ahora, estamos en condiciones de programar el algoritmo Tie and Transfer para el problema idénticamente constreñido. Este ha sido programado para números de tipo "float", por lo que en ocasiones se producen errores de redondeo que eliminan algunos empates que son necesarios, por ello, se proponen tres algoritmos distintos que varían el último paso para evitar estos errores de redondeo.

Una primera aproximación al método consiste en volver a hacer el algoritmo tantas veces como se requiera, tomando como nuevos valores del vector de votos dicho vector con los factores de corrección lambda y mu, de manera que los valores de las diagonales de los futuros lambda y mu serán muy próximos a 1, tan sólo mejorando el error produciendo nuevos empates. Empíricamente la probabilidad de que estos errores permanezcan es muy reducida (no se da con datos reales que no precisan de empates).

In [22]:
def TieAndTransfer(votos,r,c,d=lambda x:x+1):
    """
    args: (votos,r,c[,d]) donde
    votos es una lista de listas o un array bidimensional que representará la matriz de votos,
    r es una lista o array unidimensional en el que cada posición i contiene el número de escaños a repartir entre los elementos de la fila i,
    c es una lista o array unidimensional en el que cada posición j contiene el número de escaños a repartir entre los elementos de la columna j y
    d (opcional) es la función divisora que se usa para aproximar los valores a enteros. Como valor predeterminado será la correspondiente a D'Hondt..
    
    Función para obtener la solución resultante del reparto biproporcional con matriz de votos "m" y restricciones por filas y columnas "r" y "c" utilizando como función divisora "d".
    Esta función difiere de las demás funciones TieAndTransfer en que hace el algoritmo tantas veces como se requiera, tomando como nuevos valores del vector de votos dicho vector con los factores de corrección lambda y mu, de manera que los valores de las diagonales de los futuros lambda y mu serán muy próximos a 1, tan sólo mejorando el error produciendo nuevos empates. 
    
    return:(sol_final,vlambda,vmu) donde 
    sol_final es una lista que contiene los distintos arrays que son solución del reparto,
    vlambda es una lista con los factores por los que se han multiplicado las filas de la matriz de votos para obtener la matriz (o matrices) de reparto y
    vmu es una lista con los factores por los que se han multiplicado las columnas de la matriz de votos para obtener la matriz (o matrices) de reparto.
    Ejemplo:
    >>> v = [[1,2],[1,2],[1,2]]
    >>> r = [1,2,1]
    >>> c = [2,2]
    >>> s = TieAndTransfer(v,r,c)
    >>> s
    """
    v=array(votos)
    sz=v.shape
    if(len(r)!=sz[0] or len(c)!=sz[1]):
        print("Hay una incompatibilidad entre las dimensiones de v y las de r o c")
        sol_final=[]
        vlambda=[]
        vmu=[]
    else:
        vlambda=[] #vector que almacenará los valores de la diagonal de lambda
        vmu=[] #vector que almacenará los valores de la diagonal de mu
        for i in range(sz[0]):
            vlambda+=[[1]]
        for j in range(sz[1]):
            vmu+=[1]
        lambda_por_v=vlambda*v
        #paso 0
        ultimos=[]
        primeros=[]
        a1=[]
        empates=[]
        concedidos=[]
        for j in range(sz[1]):
            a0,ultimo,primero,empate,concedido=Divisores4((lambda_por_v)[:,j],c[j],d)
            ultimos+=[ultimo]
            primeros+=[primero]
            a1+=[a0]
            empates+=[empate]
            concedidos+=[concedido]
        e=transpose(array(empates))
        conce=transpose(array(concedidos))
        a=transpose(array(a1))
        r1=a.sum(axis=1)
        I0=[]
        Imas=[]
        Imenos=[]
        error=0
        for i in range(sz[0]):
            if r1[i]==r[i]:
                I0+=[i]
            elif r1[i]>r[i]:
                Imas+=[i]
                error+=(r1[i]-r[i])
            else:
                Imenos+=[i]
        grafo=construir_grafo(e,conce,Imas,Imenos,I0)
        #comprobación de si hay filas sobrerepresentadas o se pasa al último paso
        
        
        while(len(Imas)!=0):
            error=0                
            for i in Imas:
                error+=(r1[i]-r[i])
            #print("El error en esta iteración es de: ",2*error)
            #Paso 4 k + 2 y 4k + 3
            Imasmodificado=[("f"+str(elemento)) for elemento in Imas]
            Imenosmodificado=[("f"+str(elemento)) for elemento in Imenos]
            camino, comprobados=find_path_modified(grafo,Imasmodificado,Imenosmodificado)
            if camino:
                #modifico elementos por primera arista (fila sobre)
                i=int(camino[0][1:])
                j=int(camino[1][1:])
                a[i,j]=a[i,j]-1
                primeros[j][i]=ultimos[j][i]
                const=d(a[i,j]-1)
                if const<=0:
                    ultimos[j][i]=oo
                else:
                    ultimos[j][i]=lambda_por_v[i][j]/const
                conce[i,j]=0
                r1[i]=r1[i]-1
                #modifico el grafo
                if len(grafo[camino[0]])==1:
                    del(grafo[camino[0]])
                else:
                    grafo[camino[0]]=[elem for elem in grafo[camino[0]] if elem!=camino[1]]
                if r1[i]==r[i]:
                    if (camino[1] in grafo):
                        grafo[camino[1]]=grafo[camino[1]]+[camino[0]]
                    else:
                        grafo[camino[1]]=[camino[0]]
                    Imas=[elemento for elemento in Imas if elemento!=i]
                    Imasmodificado=[elemento for elemento in Imasmodificado if elemento!=("f"+str(i))]
                    I0+=[i]                    
                
                #modifico elementos por ultima arista (fila infra)
                i=int(camino[-1][1:])
                j=int(camino[-2][1:])
                a[i,j]=a[i,j]+1
                ultimos[j][i]=primeros[j][i]
                const=d(a[i,j])
                if const<=0:
                    primeros[j][i]=oo
                else:
                    primeros[j][i]=lambda_por_v[i][j]/const
                conce[i,j]=1
                r1[i]=r1[i]+1
                #modifico el grafo
                if len(grafo[camino[-2]])==1:
                    del(grafo[camino[-2]])
                else:
                    grafo[camino[-2]]=[elem for elem in grafo[camino[-2]] if elem!=camino[-1]]
                if r1[i]==r[i]:
                    if (camino[-1] in grafo):
                        grafo[camino[-1]]=grafo[camino[-1]]+[camino[-2]]
                    else:
                        grafo[camino[-1]]=[camino[-2]]
                    Imenos=[elemento for elemento in Imenos if elemento!=i]
                    Imenosmodificado=[elemento for elemento in Imenosmodificado if elemento!=("f"+str(i))]
                    I0+=[i]
                for indice in range(1,len(camino)-2):    
                    if camino[indice][0]=="f":
                        i=int(camino[indice][1:])
                        j=int(camino[indice+1][1:])
                        a[i,j]=a[i,j]-1
                        primeros[j][i]=ultimos[j][i]
                        const=d(a[i,j]-1)
                        if const<=0:
                            ultimos[j][i]=oo
                        else:
                            ultimos[j][i]=lambda_por_v[i][j]/const
                        conce[i,j]=0
                    else:
                        j=int(camino[indice][1:])
                        i=int(camino[indice+1][1:])
                        a[i,j]=a[i,j]+1
                        ultimos[j][i]=primeros[j][i]
                        const=d(a[i,j])
                        if const<=0:
                            primeros[j][i]=oo
                        else:
                            primeros[j][i]=lambda_por_v[i][j]/const
                        conce[i,j]=1
                    #modifico el grafo
                    if len(grafo[camino[indice]])==1:
                        del(grafo[camino[indice]])
                    else:
                        grafo[camino[indice]]=[elem for elem in grafo[camino[indice]] if elem!=camino[indice+1]]
                    if (camino[indice+1] in grafo):
                        grafo[camino[indice+1]]=grafo[camino[indice+1]]+[camino[indice]]
                    else:
                        grafo[camino[indice+1]]=[camino[indice]]
            #paso 4k+3 b y 4k+4
            else:
                I00=[]
                I0mas=[]
                for i in I0:
                    if ("f"+str(i)) in comprobados:
                        I0mas+=[i]
                    else:
                        I00+=[i]
                #paso 4k+4
                Imenos00=Imenos[:]+I00[:]
                Imas0mas=Imas[:]+I0mas[:]
                cociente_maximo=0
                posiciones_cociente_maximo=[]
                posiciones_minimo=[]
                posiciones_maximo=[]
                for j in range(sz[1]):
                    buscarminimo=[ultimos[j][i] for i in Imas0mas]
                    minim,num_min,pos_min=BuscaMinimo2(buscarminimo)
                    buscarmaximo=[primeros[j][i] for i in Imenos00]
                    maxim,num_max,pos_max=BuscaMaximo2(buscarmaximo)
                    cociente=maxim/minim
                    if (cociente>1):#no debería ocurrir
                        print("Hay un error de tipo 5")
                    if cociente>cociente_maximo:
                        cociente_maximo=cociente
                        posiciones_cociente_maximo=[j]
                        posiciones_minimo=[[Imas0mas[ii] for ii in pos_min]]
                        posiciones_maximo=[[Imenos00[ii] for ii in pos_max]]
                    elif cociente==cociente_maximo:
                        posiciones_cociente_maximo+=[j]
                        posiciones_minimo+=[[Imas0mas[ii] for ii in pos_min]]
                        posiciones_maximo+=[[Imenos00[ii] for ii in pos_max]]
                if cociente_maximo==0:
                        return([],[],[])
                sum_empatesconcedidos_columnas=conce.sum(axis=0)
                for j in range(sz[1]):
                    for i in Imas0mas:
                        ultimos[j][i]=ultimos[j][i]*cociente_maximo
                        primeros[j][i]=primeros[j][i]*cociente_maximo
                    empates_en_Imenos00j=0
                    for i in Imenos00:
                        empates_en_Imenos00j+=e[i][j]
                        
                    
                    if sum_empatesconcedidos_columnas[j]<empates_en_Imenos00j:
                        for i in Imas0mas:
                            if e[i][j]==1:
                                if conce[i][j]==1: #(no debería ocurrir)
                                    print("Hay un error de tipo 1 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if grafo[("f"+str(i))]==[("c"+str(j))]:
                                            del(grafo[("f"+str(i))])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else:
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if grafo[("c"+str(j))]==[("f"+str(i))]:
                                            del(grafo[("c"+str(j))])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_minimo[k]:
                                e[i][j]=1
                                conce[i][j]=1
                                if ("f"+str(i)) in grafo:
                                    grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                                else:
                                    grafo["f"+str(i)]=["c"+str(j)]
                                
                            
                            
                        
                        
                        
                        
                    elif sum_empatesconcedidos_columnas[j]>empates_en_Imenos00j:
                        for i in Imenos00:
                            if e[i][j]==1:
                                if conce[i][j]==1:
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if grafo[("f"+str(i))]==("c"+str(j)):
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else: #no debería ocurrir
                                    print("Hay un error de tipo 2 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if grafo[("c"+str(j))]==("f"+str(i)):
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_maximo[k]:
                                e[i][j]=1
                                conce[i][j]=0
                                if ("c"+str(j)) in grafo:
                                    grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                                else:
                                    grafo["c"+str(j)]=["f"+str(i)]
                                
                                    
                    else:
                        for i in Imas0mas:
                            if e[i][j]==1:
                                if conce[i][j]==1: #(no debería ocurrir)
                                    print("hay un error de tipo 3 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if len(grafo[("f"+str(i))])==1:
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else:
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if len(grafo[("c"+str(j))])==1:
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        for i in Imenos00:
                            if e[i][j]==1:
                                if conce[i][j]==1:
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if len(grafo[("f"+str(i))])==1:
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else: #no debería ocurrir
                                    print("hay un error de tipo 4 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if len(grafo[("c"+str(j))])==1:
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]

                        
                        
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_maximo[k]:
                                e[i][j]=1
                                conce[i][j]=0
                                if ("c"+str(j)) in grafo:
                                    grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                                else:
                                    grafo["c"+str(j)]=["f"+str(i)]
                            for i in posiciones_minimo[k]:
                                e[i][j]=1
                                conce[i][j]=1
                                if ("f"+str(i)) in grafo:
                                    grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                                else:
                                    grafo["f"+str(i)]=["c"+str(j)]

                            

                        
                        
                for i in Imas0mas:
                    vlambda[i][0]=vlambda[i][0]*cociente_maximo
                lambda_por_v=vlambda*v
        #paso final
        sol_final=[]
        sol_final_sin_empates_t=[]
        posiciones_empates=zeros(sz,dtype=type(0))
        c2=[]
        r2=[]
        error2=0
        for j in range(sz[1]):
            sol_final_sin_empatesj, invvmuj, posem, sobrantes=DivisoresSinEmpates(transpose(lambda_por_v)[j],c[j],d)
            sol_final_sin_empates_t+=[sol_final_sin_empatesj]
            vmu[j]=1/invvmuj
            for i in posem:
                posiciones_empates[i,j]=1
            c2+=[sobrantes]
            error2+=sobrantes
        sol_final_sin_empates=transpose(array(sol_final_sin_empates_t))
        r21=sol_final_sin_empates.sum(axis=1)
        for i in range(sz[0]):
            r2+=[r[i]-r21[i]]
        sol_final=matrix_to_problem(posiciones_empates,r2,c2,sol_final_sin_empates)
        if(sol_final==[]): #si no ha encontrado solución, es debido a los empates, por lo que buscará una solución nueva con la aproximación de vlambda*v*vmu que será muy próxima a la solución. En ocasiones pudiera ocurrir que no se llegue a la solución por errores de aproximación que eliminen empates.
            print("Buscando soluciones... \n Este proceso podría ser un bucle infinito.")
            sol_final,vlambda1,vmu1=TieAndTransfer(vlambda*v*vmu,r,c,d)
            vlambda=array(vlambda)*array(vlambda1)
            vmu=array(vmu)*array(vmu1)
    return(sol_final,vlambda,vmu)

En segundo lugar, puesto que en los pasos previos al último paso se encuentra una solución, se propone como "única solución" la encontrada advirtiendo que pudiera ocurrir que esa solución no sea única.

In [23]:
def TieAndTransfer2(votos,r,c,d=lambda x:x+1):
    """
    args: (votos,r,c[,d]) donde
    votos es una lista de listas o un array bidimensional que representará la matriz de votos,
    r es una lista o array unidimensional en el que cada posición i contiene el número de escaños a repartir entre los elementos de la fila i,
    c es una lista o array unidimensional en el que cada posición j contiene el número de escaños a repartir entre los elementos de la columna j y
    d (opcional) es la función divisora que se usa para aproximar los valores a enteros. Como valor predeterminado será la correspondiente a D'Hondt..
    
    Función para obtener la solución resultante del reparto biproporcional con matriz de votos "m" y restricciones por filas y columnas "r" y "c" utilizando como función divisora "d".
    Esta función difiere del resto de funciones TieAndTransfer en que sólo da una solución, obviando el caso que haya más, en caso de que los empates necesarios no sean exactos.
    
    return:(sol_final,vlambda,vmu) donde 
    sol_final es una lista que contiene los distintos arrays que son solución del reparto,
    vlambda es una lista con los factores por los que se han multiplicado las filas de la matriz de votos para obtener la matriz (o matrices) de reparto y
    vmu es una lista con los factores por los que se han multiplicado las columnas de la matriz de votos para obtener la matriz (o matrices) de reparto.
    Ejemplo:
    >>> v = [[1,2],[1,2],[1,2]]
    >>> r = [1,2,1]
    >>> c = [2,2]
    >>> s = TieAndTransfer2(v,r,c)
    >>> s
    """
    v=array(votos)
    sz=v.shape
    if(len(r)!=sz[0] or len(c)!=sz[1]):
        print("Hay una incompatibilidad entre las dimensiones de v y las de r o c")
        sol_final=[]
        vlambda=[]
        vmu=[]
    else:
        vlambda=[] #vector que almacenará los valores de la diagonal de lambda
        vmu=[] #vector que almacenará los valores de la diagonal de mu
        for i in range(sz[0]):
            vlambda+=[[1]]
        for j in range(sz[1]):
            vmu+=[1]
        lambda_por_v=vlambda*v
        #paso 0
        ultimos=[]
        primeros=[]
        a1=[]
        empates=[]
        concedidos=[]
        for j in range(sz[1]):
            a0,ultimo,primero,empate,concedido=Divisores4((lambda_por_v)[:,j],c[j],d)
            ultimos+=[ultimo]
            primeros+=[primero]
            a1+=[a0]
            empates+=[empate]
            concedidos+=[concedido]
        e=transpose(array(empates))
        conce=transpose(array(concedidos))
        a=transpose(array(a1))
        r1=a.sum(axis=1)
        I0=[]
        Imas=[]
        Imenos=[]
        error=0
        for i in range(sz[0]):
            if r1[i]==r[i]:
                I0+=[i]
            elif r1[i]>r[i]:
                Imas+=[i]
                error+=(r1[i]-r[i])
            else:
                Imenos+=[i]
        grafo=construir_grafo(e,conce,Imas,Imenos,I0)
        #comprobación de si hay filas sobrerepresentadas o se pasa al último paso
        
        
        while(len(Imas)!=0):
            error=0                
            for i in Imas:
                error+=(r1[i]-r[i])
            #print("El error en esta iteración es de: ",2*error)
            #Paso 4 k + 2 y 4k + 3
            Imasmodificado=[("f"+str(elemento)) for elemento in Imas]
            Imenosmodificado=[("f"+str(elemento)) for elemento in Imenos]
            camino, comprobados=find_path_modified(grafo,Imasmodificado,Imenosmodificado)
            if camino:
                #modifico elementos por primera arista (fila sobre)
                i=int(camino[0][1:])
                j=int(camino[1][1:])
                a[i,j]=a[i,j]-1
                primeros[j][i]=ultimos[j][i]
                const=d(a[i,j]-1)
                if const<=0:
                    ultimos[j][i]=oo
                else:
                    ultimos[j][i]=lambda_por_v[i][j]/const
                conce[i,j]=0
                r1[i]=r1[i]-1
                #modifico el grafo
                if len(grafo[camino[0]])==1:
                    del(grafo[camino[0]])
                else:
                    grafo[camino[0]]=[elem for elem in grafo[camino[0]] if elem!=camino[1]]
                if r1[i]==r[i]:
                    if (camino[1] in grafo):
                        grafo[camino[1]]=grafo[camino[1]]+[camino[0]]
                    else:
                        grafo[camino[1]]=[camino[0]]
                    Imas=[elemento for elemento in Imas if elemento!=i]
                    Imasmodificado=[elemento for elemento in Imasmodificado if elemento!=("f"+str(i))]
                    I0+=[i]                    
                
                #modifico elementos por ultima arista (fila infra)
                i=int(camino[-1][1:])
                j=int(camino[-2][1:])
                a[i,j]=a[i,j]+1
                ultimos[j][i]=primeros[j][i]
                const=d(a[i,j])
                if const<=0:
                    primeros[j][i]=oo
                else:
                    primeros[j][i]=lambda_por_v[i][j]/const
                conce[i,j]=1
                r1[i]=r1[i]+1
                #modifico el grafo
                if len(grafo[camino[-2]])==1:
                    del(grafo[camino[-2]])
                else:
                    grafo[camino[-2]]=[elem for elem in grafo[camino[-2]] if elem!=camino[-1]]
                if r1[i]==r[i]:
                    if (camino[-1] in grafo):
                        grafo[camino[-1]]=grafo[camino[-1]]+[camino[-2]]
                    else:
                        grafo[camino[-1]]=[camino[-2]]
                    Imenos=[elemento for elemento in Imenos if elemento!=i]
                    Imenosmodificado=[elemento for elemento in Imenosmodificado if elemento!=("f"+str(i))]
                    I0+=[i]
                for indice in range(1,len(camino)-2):    
                    if camino[indice][0]=="f":
                        i=int(camino[indice][1:])
                        j=int(camino[indice+1][1:])
                        a[i,j]=a[i,j]-1
                        primeros[j][i]=ultimos[j][i]
                        const=d(a[i,j]-1)
                        if const<=0:
                            ultimos[j][i]=oo
                        else:
                            ultimos[j][i]=lambda_por_v[i][j]/const
                        conce[i,j]=0
                    else:
                        j=int(camino[indice][1:])
                        i=int(camino[indice+1][1:])
                        a[i,j]=a[i,j]+1
                        ultimos[j][i]=primeros[j][i]
                        const=d(a[i,j])
                        if const<=0:
                            primeros[j][i]=oo
                        else:
                            primeros[j][i]=lambda_por_v[i][j]/const
                        conce[i,j]=1
                    #modifico el grafo
                    if len(grafo[camino[indice]])==1:
                        del(grafo[camino[indice]])
                    else:
                        grafo[camino[indice]]=[elem for elem in grafo[camino[indice]] if elem!=camino[indice+1]]
                    if (camino[indice+1] in grafo):
                        grafo[camino[indice+1]]=grafo[camino[indice+1]]+[camino[indice]]
                    else:
                        grafo[camino[indice+1]]=[camino[indice]]
            #paso 4k+3 b y 4k+4
            else:
                I00=[]
                I0mas=[]
                for i in I0:
                    if ("f"+str(i)) in comprobados:
                        I0mas+=[i]
                    else:
                        I00+=[i]
                #paso 4k+4
                Imenos00=Imenos[:]+I00[:]
                Imas0mas=Imas[:]+I0mas[:]
                cociente_maximo=0
                posiciones_cociente_maximo=[]
                posiciones_minimo=[]
                posiciones_maximo=[]
                for j in range(sz[1]):
                    buscarminimo=[ultimos[j][i] for i in Imas0mas]
                    minim,num_min,pos_min=BuscaMinimo2(buscarminimo)
                    buscarmaximo=[primeros[j][i] for i in Imenos00]
                    maxim,num_max,pos_max=BuscaMaximo2(buscarmaximo)
                    cociente=maxim/minim
                    if (cociente>1):#no debería ocurrir
                        print("Hay un error de tipo 5")
                    if cociente>cociente_maximo:
                        cociente_maximo=cociente
                        posiciones_cociente_maximo=[j]
                        posiciones_minimo=[[Imas0mas[ii] for ii in pos_min]]
                        posiciones_maximo=[[Imenos00[ii] for ii in pos_max]]
                    elif cociente==cociente_maximo:
                        posiciones_cociente_maximo+=[j]
                        posiciones_minimo+=[[Imas0mas[ii] for ii in pos_min]]
                        posiciones_maximo+=[[Imenos00[ii] for ii in pos_max]]
                if cociente_maximo==0:
                        return([],[],[])
                sum_empatesconcedidos_columnas=conce.sum(axis=0)
                for j in range(sz[1]):
                    for i in Imas0mas:
                        ultimos[j][i]=ultimos[j][i]*cociente_maximo
                        primeros[j][i]=primeros[j][i]*cociente_maximo
                    empates_en_Imenos00j=0
                    for i in Imenos00:
                        empates_en_Imenos00j+=e[i][j]
                        
                    
                    if sum_empatesconcedidos_columnas[j]<empates_en_Imenos00j:
                        for i in Imas0mas:
                            if e[i][j]==1:
                                if conce[i][j]==1: #(no debería ocurrir)
                                    print("Hay un error de tipo 1 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if grafo[("f"+str(i))]==[("c"+str(j))]:
                                            del(grafo[("f"+str(i))])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else:
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if grafo[("c"+str(j))]==[("f"+str(i))]:
                                            del(grafo[("c"+str(j))])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_minimo[k]:
                                e[i][j]=1
                                conce[i][j]=1
                                if ("f"+str(i)) in grafo:
                                    grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                                else:
                                    grafo["f"+str(i)]=["c"+str(j)]
                                
                            
                            
                        
                        
                        
                        
                    elif sum_empatesconcedidos_columnas[j]>empates_en_Imenos00j:
                        for i in Imenos00:
                            if e[i][j]==1:
                                if conce[i][j]==1:
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if grafo[("f"+str(i))]==("c"+str(j)):
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else: #no debería ocurrir
                                    print("Hay un error de tipo 2 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if grafo[("c"+str(j))]==("f"+str(i)):
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_maximo[k]:
                                e[i][j]=1
                                conce[i][j]=0
                                if ("c"+str(j)) in grafo:
                                    grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                                else:
                                    grafo["c"+str(j)]=["f"+str(i)]
                                
                                    
                    else:
                        for i in Imas0mas:
                            if e[i][j]==1:
                                if conce[i][j]==1: #(no debería ocurrir)
                                    print("hay un error de tipo 3 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if len(grafo[("f"+str(i))])==1:
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else:
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if len(grafo[("c"+str(j))])==1:
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        for i in Imenos00:
                            if e[i][j]==1:
                                if conce[i][j]==1:
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if len(grafo[("f"+str(i))])==1:
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else: #no debería ocurrir
                                    print("hay un error de tipo 4 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if len(grafo[("c"+str(j))])==1:
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]

                        
                        
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_maximo[k]:
                                e[i][j]=1
                                conce[i][j]=0
                                if ("c"+str(j)) in grafo:
                                    grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                                else:
                                    grafo["c"+str(j)]=["f"+str(i)]
                            for i in posiciones_minimo[k]:
                                e[i][j]=1
                                conce[i][j]=1
                                if ("f"+str(i)) in grafo:
                                    grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                                else:
                                    grafo["f"+str(i)]=["c"+str(j)]

                            

                        
                        
                for i in Imas0mas:
                    vlambda[i][0]=vlambda[i][0]*cociente_maximo
                lambda_por_v=vlambda*v
        #paso final
        sol_final=[]
        sol_final_sin_empates_t=[]
        posiciones_empates=zeros(sz,dtype=type(0))
        c2=[]
        r2=[]
        error2=0
        for j in range(sz[1]):
            sol_final_sin_empatesj, invvmuj, posem, sobrantes=DivisoresSinEmpates(transpose(lambda_por_v)[j],c[j],d)
            sol_final_sin_empates_t+=[sol_final_sin_empatesj]
            vmu[j]=1/invvmuj
            for i in posem:
                posiciones_empates[i,j]=1
            c2+=[sobrantes]
            error2+=sobrantes
        sol_final_sin_empates=transpose(array(sol_final_sin_empates_t))
        r21=sol_final_sin_empates.sum(axis=1)
        for i in range(sz[0]):
            r2+=[r[i]-r21[i]]
        sol_final=matrix_to_problem(posiciones_empates,r2,c2,sol_final_sin_empates)
        if(sol_final==[]): #si no ha encontrado solución, es debido a los empates, por lo que dará la solución conocida y dirá que podría haber más soluciones.
            print("Debido a las aproximaciones, podría haber más soluciones que no se han tenido en cuenta además de esta")
            sol_final=[a]
    return(sol_final,vlambda,vmu)

Finalmente, la última versión de este algoritmo toma como posiciones en empate las de la matriz interna que ha ido almacenando esas posiciones a lo largo del algoritmo, obteniendo así todas las posibles combinaciones de esos empates que son compatibles con las condiciones. Quizá sea esta solución la que aporte un mayor número de soluciones correctas a costa de un error ínfimo en los valores de lambda y mu, por lo que a partir de ahora se considerará esta función.

In [24]:
def TieAndTransfer3(votos,r,c,d=lambda x:x+1):
    """
    args: (votos,r,c[,d]) donde
    votos es una lista de listas o un array bidimensional que representará la matriz de votos,
    r es una lista o array unidimensional en el que cada posición i contiene el número de escaños a repartir entre los elementos de la fila i,
    c es una lista o array unidimensional en el que cada posición j contiene el número de escaños a repartir entre los elementos de la columna j y
    d (opcional) es la función divisora que se usa para aproximar los valores a enteros. Como valor predeterminado será la correspondiente a D'Hondt..
    
    Función para obtener la solución resultante del reparto biproporcional con matriz de votos "m" y restricciones por filas y columnas "r" y "c" utilizando como función divisora "d".
    Esta función difiere de las demás funciones TieAndTransfer en que toma como matriz de empates la que tiene acumulada anteriormente.
    
    return:(sol_final,vlambda,vmu) donde 
    sol_final es una lista que contiene los distintos arrays que son solución del reparto,
    vlambda es una lista con los factores por los que se han multiplicado las filas de la matriz de votos para obtener la matriz (o matrices) de reparto y
    vmu es una lista con los factores por los que se han multiplicado las columnas de la matriz de votos para obtener la matriz (o matrices) de reparto.
    Ejemplo:
    >>> v = [[1,2],[1,2],[1,2]]
    >>> r = [1,2,1]
    >>> c = [2,2]
    >>> s = TieAndTransfer3(v,r,c)
    >>> s
    """
    v=array(votos)
    sz=v.shape
    if(len(r)!=sz[0] or len(c)!=sz[1]):
        print("Hay una incompatibilidad entre las dimensiones de v y las de r o c")
        sol_final=[]
        vlambda=[]
        vmu=[]
    else:
        vlambda=[] #vector que almacenará los valores de la diagonal de lambda
        vmu=[] #vector que almacenará los valores de la diagonal de mu
        for i in range(sz[0]):
            vlambda+=[[1]]
        for j in range(sz[1]):
            vmu+=[1]
        lambda_por_v=vlambda*v
        #paso 0
        ultimos=[]
        primeros=[]
        a1=[]
        empates=[]
        concedidos=[]
        for j in range(sz[1]):
            a0,ultimo,primero,empate,concedido=Divisores4((lambda_por_v)[:,j],c[j],d)
            ultimos+=[ultimo]
            primeros+=[primero]
            a1+=[a0]
            empates+=[empate]
            concedidos+=[concedido]
        e=transpose(array(empates))
        conce=transpose(array(concedidos))
        a=transpose(array(a1))
        r1=a.sum(axis=1)
        I0=[]
        Imas=[]
        Imenos=[]
        error=0
        for i in range(sz[0]):
            if r1[i]==r[i]:
                I0+=[i]
            elif r1[i]>r[i]:
                Imas+=[i]
                error+=(r1[i]-r[i])
            else:
                Imenos+=[i]
        grafo=construir_grafo(e,conce,Imas,Imenos,I0)
        #comprobación de si hay filas sobrerepresentadas o se pasa al último paso
        
        
        while(len(Imas)!=0):
            error=0                
            for i in Imas:
                error+=(r1[i]-r[i])
            #print("El error en esta iteración es de: ",2*error)
            #Paso 4 k + 2 y 4k + 3
            Imasmodificado=[("f"+str(elemento)) for elemento in Imas]
            Imenosmodificado=[("f"+str(elemento)) for elemento in Imenos]
            camino, comprobados=find_path_modified(grafo,Imasmodificado,Imenosmodificado)
            if camino:
                #modifico elementos por primera arista (fila sobre)
                i=int(camino[0][1:])
                j=int(camino[1][1:])
                a[i,j]=a[i,j]-1
                primeros[j][i]=ultimos[j][i]
                const=d(a[i,j]-1)
                if const<=0:
                    ultimos[j][i]=oo
                else:
                    ultimos[j][i]=lambda_por_v[i][j]/const
                conce[i,j]=0
                r1[i]=r1[i]-1
                #modifico el grafo
                if len(grafo[camino[0]])==1:
                    del(grafo[camino[0]])
                else:
                    grafo[camino[0]]=[elem for elem in grafo[camino[0]] if elem!=camino[1]]
                if r1[i]==r[i]:
                    if (camino[1] in grafo):
                        grafo[camino[1]]=grafo[camino[1]]+[camino[0]]
                    else:
                        grafo[camino[1]]=[camino[0]]
                    Imas=[elemento for elemento in Imas if elemento!=i]
                    Imasmodificado=[elemento for elemento in Imasmodificado if elemento!=("f"+str(i))]
                    I0+=[i]                    
                
                #modifico elementos por ultima arista (fila infra)
                i=int(camino[-1][1:])
                j=int(camino[-2][1:])
                a[i,j]=a[i,j]+1
                ultimos[j][i]=primeros[j][i]
                const=d(a[i,j])
                if const<=0:
                    primeros[j][i]=oo
                else:
                    primeros[j][i]=lambda_por_v[i][j]/const
                conce[i,j]=1
                r1[i]=r1[i]+1
                #modifico el grafo
                if len(grafo[camino[-2]])==1:
                    del(grafo[camino[-2]])
                else:
                    grafo[camino[-2]]=[elem for elem in grafo[camino[-2]] if elem!=camino[-1]]
                if r1[i]==r[i]:
                    if (camino[-1] in grafo):
                        grafo[camino[-1]]=grafo[camino[-1]]+[camino[-2]]
                    else:
                        grafo[camino[-1]]=[camino[-2]]
                    Imenos=[elemento for elemento in Imenos if elemento!=i]
                    Imenosmodificado=[elemento for elemento in Imenosmodificado if elemento!=("f"+str(i))]
                    I0+=[i]
                for indice in range(1,len(camino)-2):    
                    if camino[indice][0]=="f":
                        i=int(camino[indice][1:])
                        j=int(camino[indice+1][1:])
                        a[i,j]=a[i,j]-1
                        primeros[j][i]=ultimos[j][i]
                        const=d(a[i,j]-1)
                        if const<=0:
                            ultimos[j][i]=oo
                        else:
                            ultimos[j][i]=lambda_por_v[i][j]/const
                        conce[i,j]=0
                    else:
                        j=int(camino[indice][1:])
                        i=int(camino[indice+1][1:])
                        a[i,j]=a[i,j]+1
                        ultimos[j][i]=primeros[j][i]
                        const=d(a[i,j])
                        if const<=0:
                            primeros[j][i]=oo
                        else:
                            primeros[j][i]=lambda_por_v[i][j]/const
                        conce[i,j]=1
                    #modifico el grafo
                    if len(grafo[camino[indice]])==1:
                        del(grafo[camino[indice]])
                    else:
                        grafo[camino[indice]]=[elem for elem in grafo[camino[indice]] if elem!=camino[indice+1]]
                    if (camino[indice+1] in grafo):
                        grafo[camino[indice+1]]=grafo[camino[indice+1]]+[camino[indice]]
                    else:
                        grafo[camino[indice+1]]=[camino[indice]]
            #paso 4k+3 b y 4k+4
            else:
                I00=[]
                I0mas=[]
                for i in I0:
                    if ("f"+str(i)) in comprobados:
                        I0mas+=[i]
                    else:
                        I00+=[i]
                #paso 4k+4
                Imenos00=Imenos[:]+I00[:]
                Imas0mas=Imas[:]+I0mas[:]
                cociente_maximo=0
                posiciones_cociente_maximo=[]
                posiciones_minimo=[]
                posiciones_maximo=[]
                for j in range(sz[1]):
                    buscarminimo=[ultimos[j][i] for i in Imas0mas]
                    minim,num_min,pos_min=BuscaMinimo2(buscarminimo)
                    buscarmaximo=[primeros[j][i] for i in Imenos00]
                    maxim,num_max,pos_max=BuscaMaximo2(buscarmaximo)
                    cociente=maxim/minim
                    if (cociente>1):#no debería ocurrir
                        print("Hay un error de tipo 5")
                    if cociente>cociente_maximo:
                        cociente_maximo=cociente
                        posiciones_cociente_maximo=[j]
                        posiciones_minimo=[[Imas0mas[ii] for ii in pos_min]]
                        posiciones_maximo=[[Imenos00[ii] for ii in pos_max]]
                    elif cociente==cociente_maximo:
                        posiciones_cociente_maximo+=[j]
                        posiciones_minimo+=[[Imas0mas[ii] for ii in pos_min]]
                        posiciones_maximo+=[[Imenos00[ii] for ii in pos_max]]
                if cociente_maximo==0:
                        return([],[],[])
                sum_empatesconcedidos_columnas=conce.sum(axis=0)
                for j in range(sz[1]):
                    for i in Imas0mas:
                        ultimos[j][i]=ultimos[j][i]*cociente_maximo
                        primeros[j][i]=primeros[j][i]*cociente_maximo
                    empates_en_Imenos00j=0
                    for i in Imenos00:
                        empates_en_Imenos00j+=e[i][j]
                        
                    
                    if sum_empatesconcedidos_columnas[j]<empates_en_Imenos00j:
                        for i in Imas0mas:
                            if e[i][j]==1:
                                if conce[i][j]==1: #(no debería ocurrir)
                                    print("Hay un error de tipo 1 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if grafo[("f"+str(i))]==[("c"+str(j))]:
                                            del(grafo[("f"+str(i))])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else:
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if grafo[("c"+str(j))]==[("f"+str(i))]:
                                            del(grafo[("c"+str(j))])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_minimo[k]:
                                e[i][j]=1
                                conce[i][j]=1
                                if ("f"+str(i)) in grafo:
                                    grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                                else:
                                    grafo["f"+str(i)]=["c"+str(j)]
                                
                            
                            
                        
                        
                        
                        
                    elif sum_empatesconcedidos_columnas[j]>empates_en_Imenos00j:
                        for i in Imenos00:
                            if e[i][j]==1:
                                if conce[i][j]==1:
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if grafo[("f"+str(i))]==("c"+str(j)):
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else: #no debería ocurrir
                                    print("Hay un error de tipo 2 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if grafo[("c"+str(j))]==("f"+str(i)):
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_maximo[k]:
                                e[i][j]=1
                                conce[i][j]=0
                                if ("c"+str(j)) in grafo:
                                    grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                                else:
                                    grafo["c"+str(j)]=["f"+str(i)]
                                
                                    
                    else:
                        for i in Imas0mas:
                            if e[i][j]==1:
                                if conce[i][j]==1: #(no debería ocurrir)
                                    print("hay un error de tipo 3 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if len(grafo[("f"+str(i))])==1:
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else:
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if len(grafo[("c"+str(j))])==1:
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]
                        for i in Imenos00:
                            if e[i][j]==1:
                                if conce[i][j]==1:
                                    e[i][j]=0
                                    conce[i][j]=0
                                    if ("f"+str(i)) in grafo:
                                        if len(grafo[("f"+str(i))])==1:
                                            del(grafo["f"+str(i)])
                                        else:
                                            grafo["f"+str(i)]=[elem for elem in grafo["f"+str(i)] if elem!=("c"+str(j))]                     
                                else: #no debería ocurrir
                                    print("hay un error de tipo 4 en el elemento de la fila ",i," y la columna ",j,".")
                                    e[i][j]=0
                                    if ("c"+str(j)) in grafo:
                                        if len(grafo[("c"+str(j))])==1:
                                            del(grafo["c"+str(j)])
                                        else:
                                            grafo["c"+str(j)]=[elem for elem in grafo["c"+str(j)] if elem!=("f"+str(i))]

                        
                        
                        if j in posiciones_cociente_maximo:
                            k=posiciones_cociente_maximo.index(j)
                            for i in posiciones_maximo[k]:
                                e[i][j]=1
                                conce[i][j]=0
                                if ("c"+str(j)) in grafo:
                                    grafo["c"+str(j)]=grafo["c"+str(j)]+["f"+str(i)]
                                else:
                                    grafo["c"+str(j)]=["f"+str(i)]
                            for i in posiciones_minimo[k]:
                                e[i][j]=1
                                conce[i][j]=1
                                if ("f"+str(i)) in grafo:
                                    grafo["f"+str(i)]=grafo["f"+str(i)]+["c"+str(j)]
                                else:
                                    grafo["f"+str(i)]=["c"+str(j)]

                            

                        
                        
                for i in Imas0mas:
                    vlambda[i][0]=vlambda[i][0]*cociente_maximo
                lambda_por_v=vlambda*v
        #paso final
        #consideraremos los empates que hay en el vector e, eliminaremos los concedidos y buscaremos una solución que cumpla las condiciones de constricción concediendo algunos de esos empates.
        posiciones_empates=e
        c2=conce.sum(axis=0)
        r2=[]
        for j in range(sz[1]):
            invvmuj,nm,pm=BuscaMinimo2(ultimos[j])
            vmu[j]=1/invvmuj
        sol_final_sin_empates=a-conce
        r21=sol_final_sin_empates.sum(axis=1)
        for i in range(sz[0]):
            r2+=[r[i]-r21[i]]
        sol_final=matrix_to_problem(array(posiciones_empates),r2,c2,sol_final_sin_empates)     
        if(sol_final==[]): #si no ha encontrado solución, es debido a los empates, por lo que dará la solución conocida y dirá que podría haber más soluciones.
            print("Hay un error.")
    return(sol_final,vlambda,vmu)

Para terminar, utilizando el reparto biproporcional se propone como algoritmo de reparto por circunscripciones un reparto biproporcional en el que el número de escaños de las circunscripciones (filas) vengan dados y el número de escaños por partido (columna) sean proporcionales (mediante alguna función divisora) al total de los votos obtenidos. La función que realiza dicho reparto es:

In [25]:
def PropuestaDeMetodo(votos,r,d1=lambda x:x+1, d2=lambda x: x+1/2):
    """
    args: (votos,r[,d1,d2]) donde
    votos es una lista de listas o un array bidimensional que representará la matriz de votos,
    r es una lista o array unidimensional en el que cada posición i contiene el número de escaños a repartir entre los elementos de la fila i y
    d1 (opcional) es la función divisora que se usa para aproximar los valores a enteros en el reparto por totales de cada columna. Como valor predeterminado será la correspondiente a D'Hondt.
    d2 (opcional) es la función divisora que se usa para aproximar los valores a enteros en el matricial. Como valor predeterminado será la correspondiente a Sainté Lague.
    
    Función para obtener la solución resultante del reparto biproporcional con matriz de votos "m" y restricciones por filas "r" utilizando como función divisora "d2".
    El valor de los escaños totales de las columnas se hará mediante un reparto por el método de divisores con función divisora "d1" de los votos totales de las columnas.
    
    return:(soluciones) donde 
    soluciones es una lista con las distintas soluciones que da el algoritmo TieAndTransfer para cada uno de los vectores de reparto de los escaños totales entre las columnas.
    
    Ejemplo:
    >>> v = [[1,2],[1,2],[1,2]]
    >>> r = [1,2,1]
    >>> s = PropuestaDeMetodo(v,r)
    >>> s
    """
    soluciones=[]
    c1=Divisores2(array(votos).sum(axis=0),array(r).sum(),d1)
    print("Los valores de la suma de escaños por columnas serán: ", c1)
    for c in c1:
        soluciones+=[TieAndTransfer3(votos,r,c,d2)]
    return(soluciones)

Compararemos las soluciones que da esta propuesta con la propuesta actual (repartos por filas), por lo que es necesario programar esta.

In [26]:
def MetodoActual(votos, r, d=lambda x: x+1):
    """
    args: (votos,r[,d]) donde
    votos es una lista de listas o un array bidimensional que representará la matriz de votos,
    r es una lista o array unidimensional en el que cada posición i contiene el número de escaños a repartir entre los elementos de la fila i y
    d (opcional) es la función divisora que se usa para aproximar los valores a enteros en los repartos por filas. Como valor predeterminado será la correspondiente a D'Hondt.
    
    Función para obtener la solución resultante del reparto por el método actual con matriz de votos "m" y restricciones por filas "r" utilizando como función divisora para los repartos por filas "d".
    
    return:(resultados) donde 
    resultados es una lista con las distintas soluciones procedentes de aplicar el método de divisores de función divisora d por filas y hacer las posibles combinaciones de todos los resultados por filas.
        
    Ejemplo:
    >>> v = [[1,2],[1,2],[1,2]]
    >>> r = [1,2,1]
    >>> s = MetodoActual(v,r)
    >>> s
    """
    v=array(votos)
    sz=v.shape
    repartosi=[]
    resultados=[[]] #Inicializamos resultados como una lista de una lista vacía
    for i in range(sz[0]): #por filas hacemos:
        repartosi=Divisores2(v[i],r[i],d) #hacemos el reparto correspondiente a los votos de esa fila
        resultados1=resultados[:] #copiamos el vector de resultados
        resultados=[] #inicializamos resultados como un vector vacío que iremos rellenando
        for rep in repartosi: #para cada elemento del reparto:
            resultados+=[res+[rep] for res in resultados1] #se añade al final de la lista de resultados los elementos de la lista resultante de añadir al final de la lista de resultados anterior los nuevos repartos.
    return(resultados)

## Ejemplos

Andalucía Marzo de 2015

In [27]:
v=array([[380093,219171,165806,91390,70067,18540,16571],
   [202785,190749,101754,79297,49625,17700,6679],
   [179843,136491,107334,59244,38018,11944,15833],
   [157229,136383,63350,43491,27709,8377,5486],
   [152263,115900,53430,32611,42387,6556,6216],
   [89369,100258,29789,25432,11376,4843,1922],
   [153424,104615,39724,21404,20552,5196,4355],
   [96272,62118,30946,17027,14692,3683,3583]]
       )
r=[18,17,15,13,11,12,11,11]

In [28]:
p=PropuestaDeMetodo(v,r)
print(p[0][0])

Los valores de la suma de escaños por columnas serán:  [[41, 30, 17, 10, 7, 2, 1]]
[array([[7, 4, 3, 2, 1, 1, 0],
       [5, 5, 3, 2, 1, 1, 0],
       [5, 4, 3, 1, 1, 0, 1],
       [5, 4, 2, 1, 1, 0, 0],
       [4, 3, 2, 1, 1, 0, 0],
       [5, 4, 2, 1, 0, 0, 0],
       [5, 3, 1, 1, 1, 0, 0],
       [5, 3, 1, 1, 1, 0, 0]])]


In [29]:
m=MetodoActual(v,r)
m

[[[8, 4, 3, 2, 1, 0, 0],
  [6, 5, 3, 2, 1, 0, 0],
  [6, 4, 3, 1, 1, 0, 0],
  [5, 4, 2, 1, 1, 0, 0],
  [5, 3, 1, 1, 1, 0, 0],
  [5, 5, 1, 1, 0, 0, 0],
  [6, 4, 1, 0, 0, 0, 0],
  [6, 3, 1, 1, 0, 0, 0]]]

Congreso de los Diputados Junio 2016

In [66]:
v=array([[1325665,678340,616503,39117,737885,0, 0, 0, 0, 0, 0, 0, 0], 
[357759, 444812, 305075, 47660, 0, 0, 0, 694315, 438126, 323824, 0, 0, 0], 
[51241, 53967, 39082, 5680, 0, 0, 0, 75328, 67759, 47226, 0, 0, 0],
[31035, 38558, 23748, 4627, 0, 0, 0, 53277, 80824, 71453, 0, 0, 0], 
[24503, 22533, 12592, 1930, 0, 0, 0, 30182, 45525, 40985, 0, 0, 0],
[482855, 285732, 205489, 18994, 0, 394227, 0, 0, 0, 0, 0, 0, 0], 
[329628, 187484, 138517, 10632, 0, 193263, 0, 0, 0, 0, 0, 0, 0], 
[106746, 66062, 44121, 2985, 0, 72281, 0, 0, 0, 0, 0, 0, 0], 
[333109, 144937, 111961, 8209, 103522, 0, 0, 0, 0, 0, 0, 0, 0], 
[163045, 93363, 67700, 7266, 118082, 0, 0, 0, 0, 0, 0, 0, 0], 
[209632, 147920, 74961, 6398, 141845, 0, 0, 0, 0, 0, 0, 0, 0], 
[299884, 347470, 138086, 13389, 214663, 0, 0, 0, 0, 0, 0, 0, 0], 
[256033, 201444, 121294, 11523, 140829, 0, 0, 0, 0, 0, 0, 0, 0], 
[198876, 175498, 88029, 9719, 130643, 0, 0, 0, 0, 0, 0, 0, 0], 
[172721, 151445, 66000, 5228, 86975, 0, 0, 0, 0, 0, 0, 0, 0], 
[153750, 139281, 55248, 4234, 85058, 0, 0, 0, 0, 0, 0, 0, 0], 
[131801, 84988, 41897, 2689, 40578, 0, 0, 0, 0, 0, 0, 0, 0], 
[131234, 138612, 38884, 3238, 53239, 0, 0, 0, 0, 0, 0, 0, 0], 
[81959, 88100, 28685, 2850, 40023, 0, 0, 0, 0, 0, 0, 0, 0], 
[170316, 119351, 65172, 6730, 113256, 0, 0, 0, 0, 0, 0, 0, 17982], 
[163129, 101120, 52576, 8243, 85178, 0, 0, 0, 0, 0, 0, 0, 60271],
[257855, 139973, 60347, 6857, 0, 0, 148566, 0, 0, 0, 0, 0, 0], 
[208921, 118816, 48302, 6497, 0, 0, 137088, 0, 0, 0, 0, 0, 0], 
[92539, 43429, 13133, 1671, 0, 0, 29136, 0, 0, 0, 0, 0, 0], 
[91516, 45796, 13343, 1939, 0, 0, 32752, 0, 0, 0, 0, 0, 0], 
[78965, 86425, 20685, 4865, 179347, 0, 0, 0, 0, 0, 175296, 67653, 0], 
[35312, 51449, 11683, 2788, 104566, 0, 0, 0, 0, 0, 85015, 69828, 0], 
[34276, 26381, 8372, 1657, 51827, 0, 0, 0, 0, 0, 26703, 15858, 0], 
[179211, 125321, 86434, 5025, 104199, 0, 0, 0, 0, 0, 0, 0, 0], 
[42332, 29915, 17934, 909, 22430, 0, 0, 0, 0, 0, 0, 0, 0], 
[30913, 19724, 9871, 402, 12558, 0, 0, 0, 0, 0, 0, 0, 0], 
[140252, 79407, 48626, 3369, 59845, 0, 0, 0, 0, 0, 0, 0, 0], 
[106976, 58173, 20505, 2757, 94972, 0, 0, 0, 0, 0, 0, 31374, 0], 
[149603, 133392, 40417, 2766, 47182, 0, 0, 0, 0, 0, 0, 0, 0], 
[95419, 78767, 24343, 1690, 33164, 0, 0, 0, 0, 0, 0, 0, 0], 
[159521, 97733, 47705, 3368, 53994, 0, 0, 0, 0, 0, 0, 0, 0], 
[121622, 81728, 33126, 2334, 37510, 0, 0, 0, 0, 0, 0, 0, 0], 
[89717, 59617, 31976, 1985, 33509, 0, 0, 0, 0, 0, 0, 0, 0], 
[53004, 34426, 10868, 785, 15263, 0, 0, 0, 0, 0, 0, 0, 0], 
[52047, 30282, 21586, 1349, 23884, 0, 0, 0, 0, 0, 0, 0, 0], 
[132026, 71780, 49283, 2225, 51637, 0, 0, 0, 0, 0, 0, 0, 0], 
[112723, 73681, 35870, 2184, 49484, 0, 0, 0, 0, 0, 0, 0, 0], 
[97672, 43252, 31878, 1359, 25441, 0, 0, 0, 0, 0, 0, 0, 0], 
[52555, 26045, 12423, 537, 15403, 0, 0, 0, 0, 0, 0, 0, 0], 
[22264, 12762, 5679, 260, 7599, 0, 0, 0, 0, 0, 0, 0, 0], 
[45955, 24751, 12428, 761, 15077, 0, 0, 0, 0, 0, 0, 0, 0], 
[88785, 45724, 30361, 1782, 35705, 0, 0, 0, 0, 0, 0, 0, 0], 
[40172, 19014, 13595, 612, 13440, 0, 0, 0, 0, 0, 0, 0, 0], 
[50931, 19277, 14096, 610, 12527, 0, 0, 0, 0, 0, 0, 0, 0], 
[73708, 42010, 24180, 1350, 28772, 0, 0, 0, 0, 0, 0, 0, 0],
[15991, 6974, 3549, 333, 3345, 0, 0, 0, 0, 0, 0, 0, 0], 
[13522, 6805, 3352, 317, 2667, 0, 0, 0, 0, 0, 0, 0, 0]]
       )
r=[36,31,6,6,4,16,12,5,10,8,8,12,11,9,7,6,6,5,5,8,7,8,7,4,4,8,6,4,7,3,3,5,5,6,4,6,5,4,3,3,5,4,4,3,2,3,4,3,3,4,1,1]

In [67]:
p=PropuestaDeMetodo(v,r)
reparto1=p[0][0][0]
reparto1

Los valores de la suma de escaños por columnas serán:  [[120, 82, 47, 4, 48, 9, 5, 12, 9, 7, 4, 2, 1]]


array([[14,  7,  7,  1,  7,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 4,  4,  3,  2,  0,  0,  0,  9,  5,  4,  0,  0,  0],
       [ 1,  1,  1,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0],
       [ 1,  1,  0,  0,  0,  0,  0,  1,  2,  1,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0],
       [ 5,  3,  2,  1,  0,  5,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  2,  2,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  1,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  2,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  2,  1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  2,  1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  4,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 4,  3,  2,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  3,  1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  2,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  2,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  2,  1,  0, 

In [70]:
a=MetodoActual(v,r)
reparto2=a[0]
array(reparto2).sum(axis=0)

array([137,  85,  32,   0,  45,   9,   5,  12,   9,   8,   5,   2,   1])

Observamos que la diferencia entre el reparto propuesto y el actual no es muy grande, siendo en la mayoría de posiciones el mismo reparto. Por tanto, la propuesta además de mejorar la proporcionalidad respecto a los votos totales de los partidos no supone un cambio radical por circunscripciones (1 o 2 escaños de diferencia).

In [30]:
array(reparto2)-reparto1

array([[ 1,  0, -1, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  1, -2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0, -1,  0, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0, 

Congreso de los Diputados Diciembre 2015

In [57]:
v=array([[117635, 89434, 44494, 39780, 0, 10828, 0, 0, 0, 0, 0, 2126, 0, 1532, 0], 
[179319, 180895, 94962, 130734, 0, 38881, 0, 0, 0, 0, 0, 7591, 0, 3637, 0], 
[142101, 149851, 55812, 68740, 0, 37650, 0, 0, 0, 0, 0, 3529, 0, 2069, 0], 
[158693, 158027, 70845, 83650, 0, 26022, 0, 0, 0, 0, 0, 3843, 0, 2697, 0], 
[74353, 95637, 30776, 39435, 0, 11786, 0, 0, 0, 0, 0, 2385, 0, 1014, 0], 
[121984, 148511, 41398, 48640, 0, 16771, 0, 0, 0, 0, 0, 2543, 0, 1287, 0], 
[224745, 208896, 132586, 132980, 0, 52772, 0, 0, 0, 0, 0, 8886, 0, 4522, 0], 
[275463, 371142, 142574, 208408, 0, 62309, 0, 0, 0, 0, 0, 9976, 0, 6421, 0], 
[39747, 30183, 19789, 21943, 0, 6433, 0, 0, 0, 0, 0, 792, 0, 833, 0], 
[28282, 19938, 11427, 11895, 0, 3897, 0, 0, 0, 0, 0, 375, 0, 456, 0], 
[161662, 118936, 95130, 102596, 0, 34869, 0, 0, 0, 0, 0, 4057, 0, 4514, 0], 
[187568, 145113, 84464, 132984, 0, 52583, 0, 0, 0, 0, 0, 4555, 0, 3855, 0], 
[140640, 88635, 71551, 111628, 0, 11451, 0, 0, 0, 0, 0, 5114, 0, 2276, 0], 
[145372, 115521, 63385, 136583, 0, 15262, 0, 0, 0, 0, 0, 5351, 0, 2564, 21788], 
[138203, 102892, 50257, 94936, 0, 15716, 0, 0, 0, 0, 0, 6537, 0, 2117, 60129], 
[129216, 78460, 53371, 62569, 0, 15488, 0, 0, 0, 0, 0, 2943, 0, 2875, 0], 
[85241, 65205, 33755, 32320, 0, 9306, 0, 0, 0, 0, 0, 1583, 0, 1146, 0], 
[113451, 91959, 36477, 36879, 0, 9834, 0, 0, 0, 0, 0, 1949, 0, 1625, 0], 
[50736, 37019, 13049, 14113, 0, 3628, 0, 0, 0, 0, 0, 537, 0, 464, 0], 
[47365, 30685, 24603, 23827, 0, 5628, 0, 0, 0, 0, 0, 1152, 0, 1026, 0], 
[149442, 106988, 53199, 52587, 0, 13590, 0, 0, 0, 0, 0, 3039, 0, 1901, 0], 
[47072, 20254, 15971, 11951, 0, 3905, 0, 0, 0, 0, 0, 519, 0, 908, 0], 
[82037, 44640, 33558, 36818, 0, 10171, 0, 0, 0, 0, 0, 1431, 0, 2356, 0], 
[104077, 74128, 37902, 51441, 0, 13997, 0, 0, 0, 0, 0, 1838, 0, 2047, 0], 
[42182, 25698, 14814, 14083, 0, 4699, 0, 0, 0, 0, 0, 624, 0, 828, 0], 
[89375, 45593, 35242, 25743, 0, 7017, 0, 0, 0, 0, 0, 1175, 0, 1669, 0], 
[36182, 19769, 15665, 13144, 0, 3742, 0, 0, 0, 0, 0, 517, 0, 1274, 0], 
[20030, 12331, 7872, 8328, 0, 1836, 0, 0, 0, 0, 0, 192, 0, 327, 0], 
[121323, 70879, 56347, 50204, 0, 17935, 0, 0, 0, 0, 0, 1735, 0, 2940, 0], 
[48160, 25985, 14608, 15865, 0, 5514, 0, 0, 0, 0, 0, 502, 0, 747, 0], 
[321980, 464588, 387061, 0, 768235, 0, 0, 414163, 378723, 0, 0, 34394, 0, 6595, 0], 
[28410, 42096, 32762, 0, 54071, 0, 0, 78030, 83170, 0, 0, 3623, 0, 586, 0], 
[22360, 24668, 17897, 0, 30538, 0, 0, 44317, 48289, 0, 0, 1743, 0, 687, 0], 
[45619, 58922, 53152, 0, 77036, 0, 0, 65272, 57071, 0, 0, 4170, 0, 686, 0], 
[14813, 7627, 4392, 4646, 0, 431, 0, 0, 0, 0, 0, 358, 0, 197, 0], 
[297083, 188711, 154397, 0, 0, 33397, 201610, 0, 0, 0, 0, 8262, 0, 5570, 0], 
[98474, 66590, 48328, 0, 0, 9605, 74732, 0, 0, 0, 0, 2252, 0, 1743, 0], 
[442578, 276188, 221896, 0, 0, 68961, 397207, 0, 0, 0, 0, 14494, 0, 9990, 0], 
[137640, 148493, 45350, 47368, 0, 12338, 0, 0, 0, 0, 0, 2041, 0, 1546, 0], 
[87924, 84758, 28406, 34730, 0, 7252, 0, 0, 0, 0, 0, 1363, 0, 1074, 0], 
[239303, 137734, 66784, 0, 0, 0, 0, 0, 0, 177581, 0, 5169, 0, 3531, 0], 
[86446, 48772, 15441, 0, 0, 0, 0, 0, 0, 39122, 0, 1633, 0, 971, 0], 
[86673, 44726, 15174, 0, 0, 0, 0, 0, 0, 34357, 0, 1406, 0, 777, 0], 
[197201, 118988, 51453, 0, 0, 0, 0, 0, 0, 159638, 0, 5315, 0, 3177, 0], 
[67941, 41973, 26812, 28073, 0, 7423, 0, 0, 0, 0, 0, 1177, 0, 1361, 0], 
[1210219, 645645, 681167, 756257, 0, 190193, 0, 0, 0, 0, 0, 28322, 0, 43508, 0], 
[12331, 6905, 4366, 3215, 0, 366, 0, 0, 0, 0, 0, 314, 0, 240, 0], 
[102244, 54856, 24969, 81216, 0, 14528, 0, 0, 0, 0, 0, 2343, 34939, 1452, 0], 
[33683, 25331, 10512, 48413, 0, 6814, 0, 0, 0, 0, 28353, 1385, 21225, 768, 0], 
[33884, 51764, 14608, 98533, 0, 10722, 0, 0, 0, 0, 91359, 2276, 81257, 1313, 0], 
[74560, 84893, 25148, 170728, 0, 18466, 0, 0, 0, 0, 182604, 4342, 81704, 2012, 0], 
[293943, 147883, 128570, 110601, 0, 22767, 0, 0, 0, 0, 6591, 0, 5442, 0, 0]])

r=[6, 9, 6, 7, 5, 5, 11, 12, 3, 3, 7, 8, 8, 8, 7, 5, 4, 5, 3, 3, 6, 3, 4, 5, 3, 4, 3, 2, 5, 3, 31, 6, 4, 6, 1, 12, 5, 15, 6, 4, 8, 4, 4, 7, 4, 36, 1, 5, 4, 6, 8, 10]

In [65]:
p=PropuestaDeMetodo(v,r)
reparto1=p[0][0][0]
array(reparto1).sum(axis=0)

Los valores de la suma de escaños por columnas serán:  [[105, 80, 50, 46, 13, 13, 9, 8, 8, 5, 4, 3, 3, 2, 1]]


array([105,  80,  50,  46,  13,  13,   9,   8,   8,   5,   4,   3,   3,
         2,   1])

In [62]:
a=MetodoActual(v,r)
reparto2=a[0]
reparto2
array(reparto2).sum(axis=0)

array([123,  90,  40,  42,  12,   2,   9,   9,   8,   6,   6,   0,   2,
         0,   1])

In [34]:
reparto1-reparto2

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1, -1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0